In [1]:
import pandas as pd

In [2]:
#Read the data into DataFrames
train = pd.read_csv('src/data/train.csv')
test = pd.read_csv('src/data/test.csv')

In [3]:
#merge the two datasets(we do not need them seperated)
df = pd.concat([train,test])

rename_mapper = {
    'boosted':'boosted',
    'full_name':'taxonomy_name',
    'free_shipping':'free_shipping',
    'conversion':'conversion',
    'is_pdp':'is_pdp',
    'logistic_type':'logistic_type',
    'original_price':'original_price',
    'platform':'platform',
    'price':'price',
    'print_position':'page_location',
    'print_server_timestamp': 'display_date',
    'title':'title'
    }

cols = list(rename_mapper.keys())

In [4]:
df['conversion'].value_counts()

False    164017
True      16744
Name: conversion, dtype: int64

In [5]:
df = df[cols].rename(rename_mapper)
df.head(5)

,boosted,full_name,free_shipping,conversion,is_pdp,logistic_type,original_price,platform,price,print_position,print_server_timestamp,title
0,False,Celulares y Teléfonos -> Celulares y Smartphones,True,False,False,cross_docking,29999,/web/desktop,20999,1717,2020-03-06T00:19:30.735-0400,Motorola One Action 128 Gb Denim Blue 4 Gb Ram
1,False,Deportes y Fitness -> Zapatillas,True,False,False,drop_off,6700,/mobile/android,3999,311,2020-04-01T21:20:11.738-0400,Zapatillas Hovr Sonic 2 Under Armour
2,False,"Ropa y Accesorios -> Pantalones, Jeans y Joggings",False,False,False,fulfillment,1490,/mobile/android,1341,1686,2020-04-08T18:38:48.360-0400,Pantalon Mujer Vestir Elastizado Tiro Alto Tal...
3,False,Juegos y Juguetes -> Vehículos de Juguete -> V...,True,False,False,fulfillment,3498,/web/mobile,3007,12,2020-04-25T22:01:19.829-0400,Trencity Pista N°3 - Kit Turbo Elevación + Pu...
4,False,Computación -> Componentes de PC -> Memorias RAM,False,False,False,cross_docking,1248,/mobile/android,998,511,2020-03-10T13:20:56.633-0400,Memoria Ram Ddr2 2gb 800mhz Samsung - Intel Y Amd


In [6]:
df['device_type'] = df['platform'].apply(lambda x: x.split('/')[2])
df['platform'] = df['platform'].apply(lambda x: x.split('/')[1])
df['taxonomy_level'] = df['full_name'].apply(lambda x: len(x.split('->'))+1)

In [7]:
for row in range(0,len(df)):
    if row%10000==0:
        print(row)
    split_tax = df.iloc[row]['full_name'].split('->')
    for i in range(0,1):
        try:
            df[f'taxonomy_level{i+1}'] = split_tax[i]
        except:
            df[f'taxonomy_level{i+1}'] = None

In [ ]:
df['conversion'].fillna(0)

In [ ]:
true_false_cols = ['free_shipping','conversion','is_pdp']
for col in true_false_cols:
    df[col] = df[col].apply(lambda x: int(x))

In [ ]:
df.to_parquet('src/data/data')